In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas as pd
import re
from itertools import compress

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import tensorflow as tf

In [ ]:
# Reading CSV file

data = pd.read_csv("../input/calculated-aqi-caaqm-central-university-hyd/Cleaned AQI.csv")

In [ ]:
TrainDf, TestDf = train_test_split(data,test_size=0.2, shuffle=False)
print(TrainDf.shape)
print(TestDf.shape)

In [ ]:
class WindowRegression():

    # featureWindow Data Blocks
    fW_XDf = pd.DataFrame()
    fW_PredictionDf = pd.DataFrame()
    fW_ResidualDf = pd.DataFrame()
    fW_RegInfo =pd.DataFrame()

    def feautureWindowRegression( self, featureSeries, DependentVariableSeries, featureSeriesTestSplit , window):
        wr= range(0,window)

        # Dependent Variable; Y for Regression
        Y = DependentVariableSeries[max(wr):featureSeriesTestSplit].values.reshape(-1,1)

        # Dependent Variable for test; using split
        Y_test = DependentVariableSeries[featureSeriesTestSplit:].values.reshape(-1,)

        # Initiating Few Data blocks for iteration
        coeffList =[]
        intList = []
        rSquareList = []
        maeList = []
        rmseList = []

        for i in wr:
            # Intermediate block to use
            Inter = featureSeries.shift(i)[max(wr):featureSeriesTestSplit]

            # Adding shifted X to object
            self.fW_XDf['Day-'+str(i)+ " X"] = Inter
            
            # X for Regression
            X = Inter.values.reshape(-1,1)

            # Values of X for Prediction
            X_test = featureSeries.values[featureSeriesTestSplit:].reshape(-1,1)

            # Regression
            reg = LinearRegression().fit(X,Y)

            # Getting info from Linear Regression
            intList.append(reg.intercept_)
            coeffList.append(reg.coef_)
            rSquareList.append(reg.score(X,Y))

            # Predicting
            Y_pred = reg.predict(X_test).reshape(-1,)

            # Getting errors from Prediction
            maeList.append(tf.keras.metrics.mean_absolute_error(y_pred=Y_pred, y_true=Y_test).numpy())
            rmseList.append(tf.sqrt(tf.losses.mean_squared_error(y_pred=Y_pred, y_true=Y_test)).numpy())


            self.fW_ResidualDf["Day-"+str(i)+" Residual"] = (Y_test - Y_pred)
            self.fW_PredictionDf["Day-"+str(i)+" Prediction"] = Y_pred
        
        coeffList = np.array(coeffList).reshape(-1,)
        intList = np.array(intList).reshape(-1,)
        rSquareList = np.array(rSquareList).reshape(-1,)
        maeList = np.array(maeList)
        rmseList = np.array(rmseList)

        self.fW_RegInfo = pd.DataFrame({"Rsquare" :rSquareList, "Intercept": intList, "Slope":coeffList, "Mean Absolute Error (MAE)":maeList,    "Root Mean Squared Error (RMSE)": rmseList}, index=wr).round(2)

        self.fW_ResidualDf = self.fW_ResidualDf.round(2)
        self.fW_PredictionDf = self.fW_PredictionDf.round(2)
        self.fW_XDf = self.fW_XDf.round(2)
    
    def plot_DayRregression(self, day):
        # Getting Line equation
        x = range(0,200)
        y = (self.fW_RegInfo["Slope"][day]*x) + self.fW_RegInfo['Intercept'][day]
        # Plotting
        plt.figure(figsize=(15,7), dpi=300, facecolor="Orange")
        plt.xlabel("(n-"+str(day)+") day value")
        plt.ylabel("n day value")

        # Scatter plot of shifted feature used for regression and Dependendent Variable
        plt.scatter(self.fW_XDf['Day-'+str(day)+ " X"], self.fW_XDf["Day-0 X"])

        # Plotting Regression Line
        plt.plot(x,y, linewidth=5, color="red")

        plt.legend(["Regression", "Scatter Points"])
        plt.title("Linear Regression with Scattered Points")
    
    def plot_WindowRegression(self):
        # Getting points from equation
        x = range(0,200)
        y = pd.DataFrame()
        for i in self.fW_RegInfo.index:
            y[str(i)] = (self.fW_RegInfo['Slope'][i]*x) + self.fW_RegInfo['Intercept'][i]
        
        # Plot
        plt.figure(figsize=(15,7), dpi=300, facecolor="cyan")
        plt.xlabel("Variable Used to predict")
        plt.ylabel("predicted value")
        plt.plot(y)
        plt.legend(y.columns)
        plt.title("Regression Lines")
    
    def plot_Predictions(self, day=None, window=None):
        if day!=None and window==None:
            plt.figure(figsize=(15,7), dpi=300, facecolor="Red")
            plt.xlabel("Points of Time (day)")
            plt.ylabel("AQI")
            plt.title("Prediction of Regression")

            plt.plot(self.fW_PredictionDf["Day-"+str(day)+" Prediction"])
            plt.legend(self.fW_PredictionDf.columns[day])
        elif day==None and window !=None:
            plt.figure(figsize=(15,7), dpi=300, facecolor="Red")
            plt.xlabel("Points of Time (day)")
            plt.ylabel("AQI")
            plt.title("Prediction of Regression")

            plt.plot(self.fW_PredictionDf[self.fW_PredictionDf.columns[window]])
            plt.legend(self.fW_PredictionDf.columns[window])
        else:
            plt.figure(figsize=(15,7), dpi=300, facecolor="Red")
            plt.xlabel("Points of Time (day)")
            plt.ylabel("AQI")
            plt.title("Predictions of Regressions")

            plt.plot(self.fW_PredictionDf)
            plt.legend(self.fW_PredictionDf.columns)

In [ ]:
# Starting an Instance of class
aqiLm = WindowRegression()

In [ ]:
# Input is (Feature Series, Dependent Series to Predict, Train Split Length, Window like 0 to 7)
aqiLm.feautureWindowRegression(data['AQI'], data['AQI'], 984, 7)

In [ ]:
# Input is (Z); Zth Day regression with Present day
aqiLm.plot_DayRregression(1)

In [ ]:
# Regression Lines of All past selfs
aqiLm.plot_WindowRegression()

In [ ]:
# Input is (window = , day =); can specify specific day to see it's prediction ; or window can be list of days you wanna see
aqiLm.plot_Predictions(window=[0,3])

In [ ]:
# column Day-1 X means 1 day past value to Y (= Day-0); similarly to other columns
# Past X values used to Predict Y values i.e (Day-0 X)
# for example, during Linear Regression, Day-1 X value tries to predict Day-0 X value
aqiLm.fW_XDf

In [ ]:
# Linear Regression info for each regression in window.
# for example, first row 0 is regression info of (Day-0 X (0 day differnce of Y) with Y which gives a X=Y line)
aqiLm.fW_RegInfo

In [ ]:
# Predictions of regression
# column Day-1 Prediction means prediction using regression of (Day-1 X(1 day past value to Y) and Y)
aqiLm.fW_PredictionDf

In [ ]:
# Residuals from Prediction
# true value - Prediction
aqiLm.fW_ResidualDf